In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
from tabulate import tabulate

In [ ]:
# HELPER
infinite = 99

def order(G):
    return len(G[0])

def generate_weight_matrix(G, weights, digraph=False):
    weight_matrix = [[0 for i in range(order(G))] for j in range (order(G))]
    for weight in weights:
        weight_matrix[weight[1]][weight[0]] = weights[weight]
        if digraph == False:
            weight_matrix[weight[0]][weight[1]] = weights[weight]
    return weight_matrix


def visualize(G, label, title = "Graph", node_color="lightblue"):
    pos = nx.circular_layout(G)

    plt.figure(figsize=(6, 4))
    nx.draw(G, pos, with_labels=True, node_color=node_color, edge_color="gray", node_size=2000, font_size=16)

    # Draw edge labels (weights)
    edge_labels = nx.get_edge_attributes(G, "weight") if label == None else label
    nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=14)
    plt.title(title, fontsize=16)
    plt.show()

def visualize_graph(G, label=None):
    G1 = nx.Graph()
    G1.add_nodes_from(G[0])
    G1.add_edges_from(G[1])
    visualize(G1, label, "Graph")
    

def visualize_digraph(G, label=None):
    G1 = nx.DiGraph()
    G1.add_nodes_from(G[0])
    G1.add_edges_from(G[1])
    visualize(G1, label, "Directed Graph")

def visualize_predecessor(G, pred,weights):
    vertex = G[0]
    edges = []
    for i in range(len(pred)):
        if (pred[i],i) in G[1]:
            edges.append((pred[i],i))
    G1 = nx.DiGraph()
    G1.add_nodes_from(vertex)
    G1.add_edges_from(edges)
    label = {}
    for w in weights:
        if w in edges:
            label[w] = weights[w]
    visualize(G1, label, "Predecessor")

def visualize_successor(G, succ,weights):
    vertex = G[0]
    edges = []
    for i in range(len(succ)):
        if (i,succ[i]) in G[1]:
            edges.append((i,succ[i]))
    G1 = nx.DiGraph()
    G1.add_nodes_from(vertex)
    G1.add_edges_from(edges)
    label = {}
    for w in weights:
        if w in edges:
            label[w] = weights[w]
    visualize(G1, label, "Successor", "lightgreen")

# Dijkstra's Algorithm

In [ ]:
# CONFIG
vertex = [0,1,2,3,4,5,6,7,8,9]
edges = [(0,1),(1,2),(1,3),(2,4),(2,5),(3,5),(4,6),(4,7),(5,9),(6,8),(7,8),(8,9)]
graph = (vertex,edges)
weights = {(0,1): 2, (1,2): 6, (1,3): 3, (2,4): 6, 
           (2,5): 1, (3,5): 3, (4,6): 1, (4,7): 1,
           (5,9): 10, (6,8): 15, (7,8): 10, (8,9):3}
visualize_digraph(graph, weights)

In [ ]:
def dijkstras(G, weights, origin, target, digraph=False):
    # Step 1
    unlabeled = G[0].copy()
    distance = [infinite for i in range(order(G))]
    distance[origin] = 0
    predeccesor = [0 for i in range(order(G))]
    labeled_vertex = []
    labeled_edges = []
    weight_matrix = generate_weight_matrix(G, weights, digraph)

    # Step 2
    unlabeled.remove(origin)
    labeled_vertex.append(origin)
    y = origin
    for i in range(order(G)):
        if weight_matrix[i][y] != 0 and distance[y]+weight_matrix[i][y] < distance[i]:
            distance[i] = distance[y]+weight_matrix[i][y]
            predeccesor[i] = y
    
    minimum_d = infinite
    for i in range(order(G)):
        if i not in labeled_vertex and distance[i] < minimum_d:
            next_y = i
            minimum_d = distance[i]
    labeled_vertex.append(next_y)
    labeled_edges.append((predeccesor[next_y],G[0][next_y], {'weight': weight_matrix[next_y][predeccesor[next_y]]}))
    if digraph:
        visualize_digraph((labeled_vertex,labeled_edges))
    else: 
        visualize_graph((labeled_vertex,labeled_edges))

    # Step 3
    while target not in labeled_vertex:
        y = next_y
        for i in range(order(G)):
            if weight_matrix[i][y] != 0 and distance[y]+weight_matrix[i][y] < distance[i]:
                distance[i] = distance[y]+weight_matrix[i][y]
                predeccesor[i] = y
        
        minimum_d = infinite
        for i in range(order(G)):
            if i not in labeled_vertex and distance[i] < minimum_d:
                next_y = i
                minimum_d = distance[i]
        labeled_vertex.append(next_y)
        labeled_edges.append((predeccesor[next_y],G[0][next_y], {'weight': weight_matrix[next_y][predeccesor[next_y]]}))
        if digraph:
            visualize_digraph((labeled_vertex,labeled_edges))
        else: 
            visualize_graph((labeled_vertex,labeled_edges))
    

dijkstras(graph, weights, 0, 9, True)

# Ford's Algorithm

In [ ]:
# CONFIG
vertex = [0,1,2]
edges = [(1,0),(1,2),(2,0)]
graph = (vertex,edges)
weights = {(1,0): 1, (1,2): 2, (2,0): -2}
visualize_digraph(graph, weights)

In [ ]:
def fords(G, origin, target, digraph=False):
    # Step 1
    unlabeled = G[0].copy()
    distance = [infinite for i in range(order(G))]
    distance[origin] = 0
    predeccesor = [0 for i in range(order(G))]
    labeled_vertex = []
    labeled_edges = []
    weight_matrix = generate_weight_matrix(G, weights, digraph)
    labels = [0 for i in range(order(G))]

    # Step 2
    unlabeled.remove(origin)
    labeled_vertex.append(origin)
    labels[origin] += 1
    y = origin
    for i in range(order(G)):
        if weight_matrix[i][y] != 0 and distance[y]+weight_matrix[i][y] < distance[i]:
            distance[i] = distance[y]+weight_matrix[i][y]
            predeccesor[i] = y
            if i in labeled_vertex:
                labeled_vertex.remove(i)
                for edge in labeled_edges:
                    if edge[0] == i or edge[1] == i:
                        labeled_edges.remove(edge)
    
    minimum_d = infinite
    for i in range(order(G)):
        if i not in labeled_vertex and distance[i] < minimum_d:
            next_y = i
            minimum_d = distance[i]
    labeled_vertex.append(next_y)
    labels[next_y] += 1
    labeled_edges.append((predeccesor[next_y],G[0][next_y], {'weight': weight_matrix[next_y][predeccesor[next_y]]}))
    if digraph:
        visualize_digraph((labeled_vertex,labeled_edges))
    else: 
        visualize_graph((labeled_vertex,labeled_edges))
    
    # Step 3
    flag = True
    while flag and order(G) not in labels:
        flag = False
        y = next_y
        for i in range(order(G)):
            if weight_matrix[i][y] != 0 and distance[y]+weight_matrix[i][y] < distance[i]:
                flag = True
                distance[i] = distance[y]+weight_matrix[i][y]
                predeccesor[i] = y
                if i in labeled_vertex:
                    labeled_vertex.remove(i)
                    for edge in labeled_edges:
                        if edge[0] == i or edge[1] == i:
                            labeled_edges.remove(edge)
        minimum_d = infinite
        for i in range(order(G)):
            if i not in labeled_vertex and distance[i] < minimum_d:
                next_y = i
                minimum_d = distance[i]
        
        flag = False if flag == False and len(labeled_vertex) == order(G) else True
        labeled_vertex.append(next_y)
        labels[next_y] += 1
        labeled_edges.append((predeccesor[next_y],G[0][next_y], {'weight': weight_matrix[next_y][predeccesor[next_y]]}))
        if digraph:
            visualize_digraph((labeled_vertex,labeled_edges))
        else: 
            visualize_graph((labeled_vertex,labeled_edges))
        

fords(graph, 1, 0, digraph=True)

# Partitioning Algorithm

In [ ]:
# CONFIG
vertex = [0,1,2,3,4,5]
edges = [(0,1),(1,2),(0,2),(2,3),(1,4),(0,5),(5,4),(4,3)]
graph = (vertex,edges)
weights = {(0,1): 4,(1,2): 3,(0,2): 7,(2,3): 2,
           (1,4): 2,(0,5): 3,(5,4): 3,(4,3): 2}
visualize_digraph(graph, weights)

In [ ]:
def partitioning(G, weights, origin, target, digraph=False):
    
    distance = [infinite for i in range(order(G))]
    distance[origin] = 0
    predecessor = [0 for i in range(order(G))]
    now = [origin]
    next = []
    weight_matrix = generate_weight_matrix(G, weights, digraph)

    u = now[0]
    now.remove(u)
    for v in range(order(G)):
        if weight_matrix[v][u] != 0 and distance[u] + weight_matrix[v][u] < distance[v]:
            distance[v] = distance[u]+weight_matrix[v][u]
            predecessor[v] = u
            next.append(v)
    visualize_predecessor(G, predecessor,weights)
    
    stop = len(next) == 0 and len(now) == 0
    while not stop:
        if len(now) == 0:
        # step 4
            for v in next:
                now.append(v)
            next = []
            stop = len(next) == 0 and len(now) == 0
        # step 2
        else: 
            u = now[0]
            now.remove(u)
            for v in range(order(G)):
                if weight_matrix[v][u] != 0 and distance[u] + weight_matrix[v][u] < distance[v]:
                    distance[v] = distance[u]+weight_matrix[v][u]
                    predecessor[v] = u
                    next.append(v)
        visualize_predecessor(G, predecessor,weights)
    return(predecessor)

partitioning(graph, weights, 0, 3, digraph=True)

# Dijkstra's Two Tree Algorithm

In [ ]:
# CONFIG
vertex = [0,1,2,3,4,5]
edges = [(0,1),(1,2),(0,2),(2,3),(1,4),(0,5),(5,4),(4,3)]
graph = (vertex,edges)
weights = {(0,1): 4,(1,2): 3,(0,2): 7,(2,3): 2,
           (1,4): 2,(0,5): 3,(5,4): 3,(4,3): 2}
visualize_digraph(graph, weights)

In [ ]:
def dijkstra_two_tree(G, weights, origin, target, digraph=False):
    # Step 1
    unlabeledS = G[0].copy()
    unlabeledT = G[0].copy()
    distanceS = [infinite for i in range(order(G))]
    distanceT = [infinite for i in range(order(G))]
    distanceS[origin] = 0
    distanceT[target] = 0
    predeccesor = [0 for i in range(order(G))]
    successor = [target for i in range(order(G))]
    labeled_vertexS = []
    labeled_vertexT = []
    labeled_edgesS = []
    labeled_edgesT = []
    weight_matrix = generate_weight_matrix(G, weights, digraph)

    # Step 2
    unlabeledS.remove(origin)
    labeled_vertexS.append(origin)
    y_s = origin
    for i in range(order(G)):
        if weight_matrix[i][y_s] != 0 and distanceS[y_s]+weight_matrix[i][y_s] < distanceS[i]:
            distanceS[i] = distanceS[y_s]+weight_matrix[i][y_s]
            predeccesor[i] = y_s
    
    minimum_d = infinite
    for i in range(order(G)):
        if i not in labeled_vertexS and distanceS[i] < minimum_d:
            next_y_s = i
            minimum_d = distanceS[i]
    labeled_vertexS.append(next_y_s)
    labeled_edgesS.append((predeccesor[next_y_s],G[0][next_y_s], {'weight': weight_matrix[next_y_s][predeccesor[next_y_s]]}))
    unlabeledS.remove(next_y_s)


    unlabeledT.remove(target)
    labeled_vertexT.append(target)
    y_t = target
    for i in range(order(G)):
        if weight_matrix[y_t][i] != 0 and distanceT[y_t]+weight_matrix[y_t][i] < distanceT[i]:
            distanceT[i] = distanceT[y_t]+weight_matrix[y_t][i]
            successor[i] = y_t
    
    minimum_d = infinite
    for i in range(order(G)):
        if i not in labeled_vertexT and distanceT[i] < minimum_d:
            next_y_t = i
            minimum_d = distanceT[i]
    labeled_vertexT.append(next_y_t)
    labeled_edgesT.append((G[0][next_y_t],successor[next_y_t], {'weight': weight_matrix[successor[next_y_t]][next_y_t]}))
    unlabeledT.remove(next_y_t)
    
    visualize_predecessor(G, predeccesor, weights)
    visualize_successor(G, successor, weights)
    # if digraph:
    #     visualize_digraph((labeled_vertex,labeled_edges))
    # else: 
    #     visualize_graph((labeled_vertex,labeled_edges))


    # Step 3
    while list(set(labeled_vertexS).union(labeled_vertexT)) != G[0]:
        y_s = next_y_s
        for i in range(order(G)):
            if weight_matrix[i][y_s] != 0 and distanceS[y_s]+weight_matrix[i][y_s] < distanceS[i]:
                distanceS[i] = distanceS[y_s]+weight_matrix[i][y_s]
                predeccesor[i] = y_s
        
        minimum_d = infinite
        for i in range(order(G)):
            if i not in labeled_vertexS and distanceS[i] < minimum_d:
                next_y_s = i
                minimum_d = distanceS[i]
        labeled_vertexS.append(next_y_s)
        labeled_edgesS.append((predeccesor[next_y_s],G[0][next_y_s], {'weight': weight_matrix[next_y_s][predeccesor[next_y_s]]}))
        unlabeledS.remove(next_y_s)


        y_t = next_y_t
        for i in range(order(G)):
            if weight_matrix[y_t][i] != 0 and distanceT[y_t]+weight_matrix[y_t][i] < distanceT[i]:
                distanceT[i] = distanceT[y_t]+weight_matrix[y_t][i]
                successor[i] = y_t
        
        minimum_d = infinite
        for i in range(order(G)):
            if i not in labeled_vertexT and distanceT[i] < minimum_d:
                next_y_t = i
                minimum_d = distanceT[i]
        labeled_vertexT.append(next_y_t)
        labeled_edgesT.append((G[0][next_y_t],successor[next_y_t], {'weight': weight_matrix[successor[next_y_t]][next_y_t]}))
        unlabeledT.remove(next_y_t)


        visualize_predecessor(G, predeccesor, weights)
        visualize_successor(G, successor, weights)
        # if digraph:
        #     visualize_digraph((labeled_vertex,labeled_edges))
        # else: 
        #     visualize_graph((labeled_vertex,labeled_edges))

    
    # Step 5
    final_distance = [infinite for i in range(order(G))]
    potential_bridge = list((set(G[0])-set(unlabeledS)).union(set(G[0])-set(unlabeledT)))
    for i in potential_bridge:
        final_distance[i] = distanceS[i]+distanceT[i]
    bridges = []
    for i in potential_bridge:
        if final_distance[i] == min(final_distance):
            bridges.append(i)
    
    t = bridges[0]
    solution = [0 for i in range(order(G))]
    while t != origin:
        solution[t] = predeccesor[t]
        t = predeccesor[t]
    t = bridges[0]
    while t != target:
        solution[successor[t]] = t
        t = successor[t] 
    visualize_predecessor(G, solution, weights)


    

dijkstra_two_tree(graph, weights, 0, 3, True)

# Floyd Shortest-Path Algorithm

In [ ]:
# CONFIG
vertex = [0,1,2,3]
edges = [(0,1),(0,2),(0,3),(1,0),(1,2),(2,0),(2,1),(2,3),(3,0),(3,2)]
graph = (vertex,edges)
weights = {(0,1): 1,(0,2): 2,(0,3): 1,
           (1,0): 2,(1,2): 7,
           (2,0): 6,(2,1): 5,(2,3): 2,
           (3,0): 1,(3,2): 4}
visualize_digraph(graph, weights)

In [ ]:
def floyd(G, weights, digraph=False):
    penut = [[i for j in range(order(G))] for i in range(order(G))]
    D_0 = [[infinite for i in range(order(G))] for j in range (order(G))]
    D_k = [[0 for i in range(order(G))] for j in range (order(G))]
    for i in range(order(G)):
        for j in range(order(G)):
            if (i,j) in weights:
                D_0[i][j] = weights[(i,j)]
            elif i == j:
                D_0[i][j] = 0
    
    k = 1
    D_l = D_0.copy()
    while k <= order(G):
        print(f"D_{k-1}:")
        print(tabulate(D_l))
        for i in range(order(G)):
            for j in range(order(G)):
                if D_l[i][k-1]+D_l[k-1][j] < D_l[i][j]:
                    D_k[i][j] = D_l[i][k-1]+D_l[k-1][j]
                    penut[i][j] = penut[k-1][j]
                else: 
                    D_k[i][j] = D_l[i][j] 
        D_l = D_k.copy()
        k += 1
    print(f"D_{k-1}:")
    print(tabulate(D_l))
    print(f"Penultimate Matrix:")
    print(tabulate(penut))
    # return (D_l,penut)
floyd(graph, weights, digraph=True)
    

# Dantzig Shortest-Path Algorithm

In [ ]:
# CONFIG
vertex = [0,1,2,3]
edges = [(0,1),(0,2),(0,3),(1,0),(1,2),(2,0),(2,1),(2,3),(3,0),(3,2)]
graph = (vertex,edges)
weights = {(0,1): 1,(0,2): 2,(0,3): 1,
           (1,0): 2,(1,2): 7,
           (2,0): 6,(2,1): 5,(2,3): 2,
           (3,0): 1,(3,2): 4}
visualize_digraph(graph, weights)

In [ ]:
def dantzig(G, weights, digraph=False):
    penut = [[i for j in range(order(G))] for i in range(order(G))]
    D_0 = [[infinite for i in range(order(G))] for j in range (order(G))]
    for i in range(order(G)):
        for j in range(order(G)):
            if (i,j) in weights:
                D_0[i][j] = weights[(i,j)]
            elif i == j:
                D_0[i][j] = 0
    
    k = 1
    D_l = D_0.copy()
    while k <= order(G):
        D_k = [[0 for i in range(k)] for j in range(k)]
        print(f"D_{k-1}:")
        print(tabulate(D_l))
        
        for j in range(k-1):
            d_j = []
            for i in range(k-1):
                d_j.append(D_0[k-1][i]+D_l[i][j])
            D_k[k-1][j] = min(d_j)
        for i in range(k-1):
            d_i = []
            for j in range(k-1):
                d_i.append(D_0[j][k-1]+D_l[i][j])
            D_k[i][k-1] = min(d_i)
        
        for i in range(k-1):
            for j in range(k-1):
                if D_k[i][k-1]+D_k[k-1][j] < D_l[i][j]:
                    D_k[i][j] = D_k[i][k-1]+D_k[k-1][j]
                else: 
                    D_k[i][j] = D_l[i][j] 
        D_l = D_k.copy()
        k += 1

    for i in range(order(G)):
        for j in range(order(G)):
            for l in range(order(G)):
                if i != l and j != l and D_l[i][l]+D_0[l][j] == D_l[i][j]:
                    penut[i][j] = l
                    
                    

    print(f"D_{k-1}:")
    print(tabulate(D_l))
    print(f"Penultimate Matrix:")
    print(tabulate(penut))
    # return (D_l,penut)
dantzig(graph, weights, digraph=True)


# Double Sweep Algorithm

In [ ]:
# HELPER

def k_add(a,b):
    k = len(a)
    res = sorted(list(set(a).union(b)))[:k]
    while len(res) < k:
        res.append(infinite)
    return res

def k_times(a,b):
    k = len(a)
    res = []
    for i in a:
        for j in b:
            if i == infinite or j == infinite:
                res.append(infinite)
            else:
                res.append(i+j)
    res = sorted(res)[:k]
    while len(res) < k:
        res.append(infinite)
    return res

def convolve(d):
    k = len(d)
    for i in range(k):
        d = k_times(d,d)
    return d
    

In [ ]:
# CONFIG

graph = [
    [[0,99,99],[1,99,99],[99,99,99],[99,99,99]],
    [[2,4,99],[0,99,99],[-1,99,99],[99,99,99]],
    [[2,99,99],[99,99,99],[0,99,99],[-1,99,99]],
    [[99,99,99],[3,99,99],[2,99,99],[0,99,99]],
]

In [ ]:
def double_sweep(graph, k, source):
    orde = len(graph)
    L = [[[infinite for a in range(k)] for i in range(orde)] for j in range(orde)]
    U = [[[infinite for a in range(k)] for i in range(orde)] for j in range(orde)]

    distance_0 = [graph[source][i] for i in range(orde)]
    distance_odd = distance_0.copy()
    distance_even = distance_0.copy()

    for i in range(orde):
        for j in range(orde):
            if i > j:
                L[i][j] = graph[i][j]
            elif i < j:
                U[i][j] = graph[i][j]


    for a in range(orde-1, -1, -1):
        d = [infinite for i in range(k)]
        d = k_add(d, distance_0[a])
        for b in range(a,orde):
            d = k_add(d, k_times(distance_odd[b],L[b][a]))
        distance_odd[a] = d
    
    for a in range(orde):
        d = [infinite for i in range(k)]
        d = k_add(d, distance_odd[a])
        for b in range(0,a):
            d = k_add(d, k_times(distance_even[b],U[b][a]))
        distance_even[a] = d
    counter = 0
    
    flag = True
    for i in range(len(distance_even)):
        if distance_even[i] != distance_odd[i]:
            flag = False

    while flag == False:
        print("Backward Sweep:")
        print(distance_odd)
        print("Forward Sweep:")
        print(distance_even)
        counter += 1
        for a in range(orde-1, -1, -1):
            d = [infinite for i in range(k)]
            d = k_add(d, distance_even[a])
            for b in range(a,orde):
                d = k_add(d, k_times(distance_odd[b],L[b][a]))
            distance_odd[a] = d
        
        for a in range(orde):
            d = [infinite for i in range(k)]
            d = k_add(d, distance_odd[a])
            for b in range(0,a):
                d = k_add(d, k_times(distance_even[b],U[b][a]))
            distance_even[a] = d
        
        flag = True
        for i in range(len(distance_even)):
            if distance_even[i] != distance_odd[i]:
                flag = False
    
    print("Backward Sweep:")
    print(distance_odd)
    print("Forward Sweep:")
    print(distance_even)
double_sweep(graph, 3, 0)


# Generalized-Floyd

In [ ]:
def g_floyd(graph, k):
    orde = len(graph)
    D_0 = graph.copy()
    D_k = [[[infinite for a in range(k)] for i in range(orde)] for j in range (orde)]
    
    k = 1
    D_l = D_0.copy()
    while k <= orde:
        print(f"D_{k-1}:")
        print(tabulate(D_l))
        for i in range(orde):
            D_k[i][i] = convolve(D_l[i][i])
        for i in range(orde):
            if i != k-1:
                D_k[i][k-1] = k_times(D_l[i][k-1],D_k[k-1][k-1])
                D_k[k-1][i] = k_times(D_k[k-1][k-1], D_l[k-1][i])
        for i in range(orde):
            for j in range(orde):
                if j != k-1:
                    D_k[i][j] = k_add(k_times(D_k[i][k-1], D_k[k-1][j]), D_l[i][j])
        D_l = D_k.copy()
        k += 1
    print(f"D_{k-1}:")
    print(tabulate(D_l))
    # print(tabulate(D_0))
    # print(tabulate(D_k))
g_floyd(graph, 3)

# Generalized Dantzig

In [ ]:
def g_dantzig(graph, k):
    orde = len(graph)
    D_0 = graph.copy()
    
    p = 1
    D_l = D_0.copy()
    while p <= orde:
        D_p = [[[infinite for a in range(k)] for i in range(p)] for j in range(p)]
        print(f"D_{p-1}:")
        print(tabulate(D_l))
        

        for a in range(p):
            d = D_0[p-1][p-1]
            for i in range(p-1):
                for j in range(p-1):
                    d = k_add(d, k_times(k_times(D_0[p-1][i],D_l[i][j]), D_0[j][p-1]))
            D_p[a][a] = convolve(d)

        for i in range(p-1):
            d = [infinite for a in range(k)]
            for j in range(p-1):
                d = k_add(d, k_times(k_times(D_p[p-1][p-1],D_0[p-1][j]), D_l[j][i]))
            D_p[p-1][i] = d

        for i in range(p-1):
            d = [infinite for a in range(k)]
            for j in range(p-1):
                d = k_add(d, k_times(k_times(D_l[i][j],D_0[j][p-1]), D_p[p-1][p-1]))
            D_p[i][p-1] = d

        for i in range(p-1):
            for j in range(p-1):
                D_p[i][j] = k_add(D_l[i][j], k_times(D_p[i][p-1], D_p[p-1][j]))
        
        D_l = D_p.copy()
        p += 1
                    
                    

    print(f"D_{k-1}:")
    print(tabulate(D_l))
    # return (D_l,penut)
g_dantzig(graph, 3)